# TP3 - *Latent Dirichlet Allocation* et Inférence variationnelle 

### Estimation avancée - G3 SDIA

Dans ce TP, on s'intéresse à la méthode "inférence variationnelle" (VI) qui permet d'approcher la loi a posteriori d'un modèle (généralement inconnue) par une autre loi plus simple (généralement un produit de lois bien connues). Nous allons l'appliquer à un modèle probabiliste pour des données textuelles, appelé *Latent Dirichlet Allocation* (LDA, qui n'a rien à voir avec la LDA *Linear Discriminant Analysis* du cours de ML).

### Instructions

1. Renommer votre notebook sous la forme `tp3_Nom1_Nom2.ipynb`, et inclure le nom du binôme dans le notebook. 

2. Votre code, ainsi que toute sortie du code, doivent être commentés !

3. Déposer votre notebook sur Moodle dans la section prévue à cet effet avant la date limite : 23 Décembre 2023, 23h59.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pickle as pkl

## Partie 0 - Introduction

LDA is a popular probabilistic model for text data, introducted in [Blei et al. (2003)](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf). In this model, the posterior distribution is intractable, and we choose to resort to variational inference (note that a Gibbs sampler would be feasible as well, but would be very slow). In particular, the CAVI updates can be easily derived.

In a few words, in LDA, each document is a mixture of topics, and each topic is a mixture of words. Uncovering those is the goal of *topic modeling*, and this is what we are going to do today. We will be using a collection of abstracts of papers published in JMLR (*Journal of Machine Learning Research*), one of the most prominent journals of the field.

**Check the .pdf file describing the model.**
The posterior is :
$$p(\boldsymbol{\beta}, \boldsymbol{\theta}, \mathbf{z} | \mathcal{D}),$$
which we are going to approximate in the following way :
$$\simeq \left[ \prod_{k=1}^K q(\beta_k) \right] \left[ \prod_{d=1}^D q(\theta_d) \right] \left[ \prod_{d=1}^D \prod_{n=1}^{N_d} q(z_{dn}) \right], $$
with :
* $q(\beta_k)$ a Dirichlet distribution (of size V) with parameter $[\lambda_{k1}, ...,\lambda_{kV}]$
* $q(\gamma_d)$ a Dirichlet distribution (of size K) with parameter $[\gamma_{d1}, ...,\gamma_{dK}]$
* $q(z_{dn})$ a Multinomial distribution (of size K) with parameter $[\phi_{dn1}, ..., \phi_{dnK}]$

The updates are as follows :
* $$\lambda_{kv} = \eta + \sum_{d=1}^D \sum_{n=1}^{N_d} w_{dnv} \phi_{dnk} $$
* $$\gamma_{dk} = \alpha + \sum_{n=1}^{N_d} \phi_{dnk}$$
* $$ \phi_{dnk} \propto \exp \left( \Psi(\gamma_{dk}) + \Psi(\lambda_{k, w_{dn}}) - \Psi(\sum_{v=1}^V \lambda_{kv}) \right)$$

$\Psi$ is the digamma function, use `scipy.special.digamma`.

## Partie 1 - Les données

The data is already prepared, see code below. We have a total of 1898 abstracts.

In [2]:
jmlr_papers = pkl.load(open("jmlr.pkl","rb")) 

**Q1.** Fill in a list of keywords from the course, to see how many papers are about probabilistic ML.

In [3]:
bayesian_jmlr_papers = []

for paper in jmlr_papers:
    bayesian_keywords = ['Bayesian', 'Bayes', 'Gaussian process', 'Markov chain Monte Carlo', 'MCMC', 'Variational inference']
    if any([kwd in paper["abstract"] for kwd in bayesian_keywords]):
        bayesian_jmlr_papers.append(paper)
        
print("There are", str(len(bayesian_jmlr_papers))+" Bayesian papers out of", str(len(jmlr_papers)))

There are 314 Bayesian papers out of 1898


Let us now preprocess the data. It is important to remove so-called "stop-words" like a, is, but, the, of, have... Scikit-learn will do the job for us. We will keep only the top-1000 words from the abstracts.

As a result, we get the count matrix $\mathbf{C}$ of size $D = 1898 \times V = 1000$. $c_{dv}$ is the number of occurrences of word $v$ in document $d$. This compact representation is called "bag-of-words". Of course from $\mathbf{C}$ you easily recover the words, since in LDA the order does not matter.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 1000, stop_words='english')
X = vectorizer.fit_transform([paper["abstract"] for paper in jmlr_papers])
print(vectorizer.get_feature_names_out()) # Top-1000 words
C = X.toarray() # Count matrix

# Removing documents with 0 words
idx = np.where(np.sum(C, axis = 1)==0)
C = np.delete(C, idx, axis = 0)

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


['100' '16' '17' '18' '949' '_blank' 'ability' 'able' 'abs' 'according'
 'account' 'accuracy' 'accurate' 'achieve' 'achieved' 'achieves' 'action'
 'actions' 'active' 'adaboost' 'adaptive' 'addition' 'additional'
 'additive' 'address' 'advantage' 'advantages' 'agent' 'aggregation' 'al'
 'algorithm' 'algorithmic' 'algorithms' 'allow' 'allowing' 'allows'
 'alternative' 'analysis' 'analyze' 'applicable' 'application'
 'applications' 'applied' 'apply' 'applying' 'approach' 'approaches'
 'appropriate' 'approximate' 'approximately' 'approximation'
 'approximations' 'arbitrary' 'art' 'article' 'artificial' 'associated'
 'assume' 'assumed' 'assumption' 'assumptions' 'asymptotic'
 'asymptotically' 'attributes' 'available' 'average' 'averaging' 'bandit'
 'base' 'based' 'basic' 'basis' 'batch' 'bayes' 'bayesian' 'behavior'
 'belief' 'benchmark' 'best' 'better' 'bias' 'bib' 'binary' 'block'
 'boosting' 'bound' 'bounded' 'bounds' 'br' 'build' 'building' 'called'
 'capture' 'carlo' 'case' 'cases' 'ca

**Q2.** How many elements of $\mathbf{C}$ are non-zero ? Is this surprising ?

In [5]:
#count the number of non-zero in C
print("There are", str(np.count_nonzero(C))+" non-zero entries in C")
print("It represents {:.2f}% of the matrix".format(np.count_nonzero(C)/C.size*100))

There are 85804 non-zero entries in C
It represents 4.53% of the matrix


> This sparsity results from the large, diverse vocabulary and the specific usage of words in academic texts. The bag-of-words model, focusing on word frequencies and the removal of common stop words, further contributes to this sparsity. The wide range of topics in JMLR leads to varied word usage across documents, accentuating the sparsity in the matrix.

## Partie 2 - Inférence variationnelle

> As you know from the lecture, VI aims at maximizing the ELBO. I have prepared for you the function to compute the ELBO.

In [6]:
from scipy.special import digamma, loggamma

def ELBO(L, G, phi, a, e, W):
    # Computes the ELBO with the values of the parameters L (Lambda), G (Gamma), and Phi
    # a, e are hyperparameters (alpha and eta)
    # W are the words (obsereved)
    
    # L - K x V matrix (variational parameters Lambda)
    # G - D x K matrix (variational parameters Gamma)
    # phi - List of D elements, each element is a Nd x K matrix (variational parameters Phi)
    # a - Scalar > 0 (hyperparameter alpha)
    # e - Scalar > 0 (hyperparameter eta)
    # W - List of D elements, each element is a Nd x V matrix (observed words)
    
    e_log_B = (digamma(L).T - digamma(np.sum(L, axis = 1))).T
    e_log_T = (digamma(G).T - digamma(np.sum(G, axis = 1))).T
    D,K=G.shape
    V=L.shape[1]
    t1 = (e-1)*np.sum(e_log_B)
    t2 = (a-1)*np.sum(e_log_T)

    phi_s = np.zeros((D,K))
    for d in range(0,D):
        phi_s[d,:] = np.sum(phi[d], axis = 0)
    t3 = np.sum(e_log_T*phi_s)
    
    tmp = np.zeros((K,V))
    for d in range(0,D):
        tmp = tmp + np.dot(phi[d].T, W[d])
    t4 = np.sum(e_log_B*tmp)
    
    t5 = np.sum(loggamma(np.sum(L, axis = 1))) - np.sum(loggamma(L)) + np.sum((L-1)*e_log_B)
    t6 = np.sum(loggamma(np.sum(G, axis = 1))) - np.sum(loggamma(G)) + np.sum((G-1)*e_log_T)

    t7 = 0
    for d in range(0,D):
        t7 = t7 + np.sum(phi[d]*np.log(phi[d] + np.spacing(1)))

    return t1 + t2 + t3 + t4 - t5 - t6 - t7

**Q1.** Transform the matrix $\mathbf{C}$ into the observed words $\mathbf{w}$. $\mathbf{w}$ should be a list of $D$ elements, each element of the list being a $N_d \times V$ matrix.

In [7]:
#transform  the matrix C into the observed words w
def transform_C_to_W(C):
    """
    Transform a word count matrix C into a list of word matrices w.

    :param C: a DxV matrix where C[d, v] is the count of word v in document d.
    :return: A list of D elements, each a N_d x V matrix representing the words in document d.
    """
    D, V = C.shape
    w = []

    for d in range(D):
        N_d = np.sum(C[d, :]) # Total number of words in document d
        word_matrix = np.zeros((N_d, V), dtype=int) # Initialize the word matrix for document d

        # Fill in the word occurrences
        word_idx = 0
        for v in range(V):
            count = C[d, v]
            for _ in range(count):
                word_matrix[word_idx, v] = 1
                word_idx += 1

        w.append(word_matrix) # Append the word matrix to the list
    return w

w = transform_C_to_W(C)
# Display the resulting word matrices
print(w)

def percentage_zeros_w(w):
    total_elements = sum(matrix.size for matrix in w)
    total_zeros = sum(np.count_nonzero(matrix == 0) for matrix in w)
    return (total_zeros / total_elements) * 100

print(percentage_zeros_w(w))


## Check 
print("On vérifie que la 1ère ligne de C est égale à la somme de la 1ere ligne de W : ", np.sum(C[0]) ==  np.sum(np.sum(w[0],axis=1)), ", et vaut :",np.sum(C[0]) )

def verify_transformation(C, W):
    # Check if the number of documents matches
    if len(C) != len(W):
        return False, "Number of documents does not match."

    for doc_idx, (doc_c, doc_w) in enumerate(zip(C, W)):
        # Total word count in the document from C
        total_words_c = np.sum(doc_c)

        # Total word count in the document from W
        total_words_w = doc_w.shape[0]

        # Check if total word counts match
        if total_words_c != total_words_w:
            return False, f"Total word count mismatch in document {doc_idx}."

        # Check if word frequencies match
        for word_idx, word_count in enumerate(doc_c):
            if word_count != np.sum(doc_w[:, word_idx]):
                return False, f"Word frequency mismatch in document {doc_idx} for word index {word_idx}."

    return True, "Transformation is correct."

# Verify the transformation
print(verify_transformation(C, w))


[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
   

**Q2.** Implement the CAVI algorithm. The updates are given at the beginning of the notebook. Monitor the convergence with the values of the ELBO (but start with a fixed number of iterations, like 50).

In [10]:
from tqdm import tqdm
def CAVI(W, K, a, e, seed):
    np.random.seed(seed)  # Set random seed for reproducibility
    D = len(W)  # Number of documents
    V = W[0].shape[1]  # Vocabulary size

    # Initialize variational parameters
    L = np.random.rand(K, V)   # Lambda - K x V
    G = 1/K*np.ones((D,K))   # Gamma - D x K
    phi = [np.zeros((W[d].shape[0],K)) for d in range(0,D)]  # Phi - List of D Nd x K matrices
    elbo_list=[]
    max_iter = 50  # Set a fixed number of iterations

    for i in tqdm(range(max_iter)):
        for d in range(D):
            Nd = W[d].shape[0]
            for n in range(Nd):
                phi_update = digamma(G[d,:]) - digamma(np.sum(L, axis = 1)) +digamma(L[:,np.where(W[d][n,:]==1)[0]].T) 
                phi[d][n, :] = np.exp(phi_update)
                phi[d][n, :] /= np.sum(phi[d][n, :])  # Normalize
                
            G[d, :] =a + np.sum(phi[d], axis = 0)
                
        for k in range(K):
            L[k, :] = e + np.sum([np.dot(W[d].T, phi[d][:,k]) for d in range(0,D)], axis = 0)
        elbo=ELBO(L, G, phi, a, e, W)
        elbo_list.append(elbo)
    print(elbo)
    return L, G, phi, elbo_list

**Q3.** Run the algorithm with $K = 10$, $\alpha = 0.5$, $\eta = 0.1$. From the results, compute the MMSE of $\lambda_{kv}$ and $\gamma_{dk}$.

**Bonus** : Re-run the algorithm several times with different initializations, and keep the solution which returns the highest ELBO.

NB : In my implementation, one iteration of the CAVI algorithm takes about 4 seconds to run.

In [11]:
K,alpha,eta = 10,0.5,0.1
seed =42
# Running the CAVI algorithm with the corrected implementation
lambda_mmse, gamma_mmse, phi_mmse,elbo_list = CAVI(w, K, alpha, eta, seed)

# Displaying a portion of the MMSE estimates of lambda and gamma
lambda_mmse[:5], gamma_mmse[:5]  # Showing the first 5 rows for brevity 

100%|██████████| 50/50 [05:17<00:00,  6.35s/it]

-735568.0198208145


(array([[ 0.10000671,  0.10000911,  0.1000069 , ...,  0.10003935,
         12.50768267,  0.10002145],
        [ 0.100006  ,  0.10001607,  0.10000588, ...,  0.10002878,
          0.10002653,  0.10004146],
        [ 0.10000977,  0.10002698,  0.10000628, ...,  0.10003936,
          0.10004494, 17.8440665 ],
        [ 0.10000761, 46.09978008,  0.10000819, ...,  0.10002035,
         26.25195678,  0.10003497],
        [ 0.10000679,  0.10001357,  0.10000483, ...,  0.10002751,
         23.54014278, 35.72820022]]),
 array([[ 5.49439474,  0.61086968, 12.20792458,  3.99961019,  1.10895762,
         20.00222314,  0.75291176,  4.93088198,  3.67395101, 10.21827529],
        [ 4.33471045,  0.6298793 ,  2.91261609,  2.44727937,  1.45309445,
         21.35867972,  1.51821376,  0.5548628 ,  3.34860355,  2.44206052],
        [10.83092077, 14.84835561,  0.63879722,  0.58085656,  2.25417587,
          4.94205685,  3.82845649, 12.0077235 ,  0.62320005,  9.44545709],
        [ 0.80221874,  7.81185834,  5.895

In [13]:
#try with different initialisation
# Running the CAVI algorithm with the corrected implementation
lambda_mmse2, gamma_mmse2, phi_mmse2,elbo_list2 = CAVI(w, 20, alpha, eta, seed)
lambda_mms3, gamma_mmse3, phi_mmse3,elbo_list3 = CAVI(w, 5, alpha, eta, seed)
lambda_mmse4, gamma_mmse4, phi_mmse4,elbo_list4 = CAVI(w, K, 0.1, eta, seed)
lambda_mmse5, gamma_mmse5, phi_mmse5,elbo_list5 = CAVI(w, K, alpha, 0.5, seed)



  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [07:56<00:00,  9.54s/it]


-724474.7455634094


100%|██████████| 50/50 [03:44<00:00,  4.48s/it]


-747315.2178497461


100%|██████████| 50/50 [08:32<00:00, 10.25s/it]


-706327.1441258168


100%|██████████| 50/50 [05:06<00:00,  6.14s/it]

-770717.2727594624


In [17]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(elbo_list)), y=elbo_list,
                    mode='lines',
                    name='1'))
                    
fig.add_trace(go.Scatter(x=np.arange(len(elbo_list2)), y=elbo_list2,mode='lines',name='2'))
fig.add_trace(go.Scatter(x=np.arange(len(elbo_list3)), y=elbo_list3,mode='lines',name='3'))
fig.add_trace(go.Scatter(x=np.arange(len(elbo_list4)), y=elbo_list4,mode='lines',name='4'))
fig.add_trace(go.Scatter(x=np.arange(len(elbo_list5)), y=elbo_list5,mode='lines',name='5'))
fig.update_layout(title='ELBO',xaxis_title='iteration',yaxis_title='ELBO')


fig.show()

**Q4.** Based on the MMSE estimates :
* What are the top-10 words per topic ? With your machine learning knowledge, can you make sense of some of the topics ?
* Choose one document at random and display its topic proportions. Comment.

In [26]:
K, V = lambda_mmse4.shape
top_words = []

for k in range(K):
    top_word_indices = np.argsort(lambda_mmse4[k, :])[-10:][::-1]
    top_word_probs = lambda_mmse4[k, top_word_indices]
    top_word=vectorizer.get_feature_names_out()[top_word_indices]
    print(f"Topic {k+1}:")
    print(f"Top words: {top_word}")




Topic 1:
Top words: ['based' 'algorithm' 'method' 'set' 'approach' 'clustering' 'new'
 'efficient' 'results' 'em']
Topic 2:
Top words: ['number' 'learning' 'problem' 'error' 'method' 'time' 'policy' 'large'
 'markov' 'size']
Topic 3:
Top words: ['sub' 'sup' 'learning' 'graph' 'model' 'regression' 'function' 'problems'
 'time' 'efficient']
Topic 4:
Top words: ['matrix' 'estimator' 'rank' 'optimal' 'information' 'lasso' 'bound'
 'sample' 'setting' 'propose']
Topic 5:
Top words: ['algorithm' 'based' 'data' 'class' 'problem' 'new' 'used' 'different'
 'paper' 'parameters']
Topic 6:
Top words: ['models' 'model' 'data' 'bayesian' 'em' 'inference' 'networks' 'learning'
 'variables' 'network']
Topic 7:
Top words: ['learning' 'nbsp' 'algorithms' 'href' 'machine' 'papers' 'font' 'pdf'
 'bib' '17']
Topic 8:
Top words: ['algorithm' 'algorithms' 'learning' 'problem' 'convex' 'vector' 'loss'
 'used' 'paper' 'results']
Topic 9:
Top words: ['data' 'method' 'methods' 'selection' 'analysis' 'linear' 'dim

I would say that the first topic is mainly about the way we can use the EM-algorithm to do clustering.

The topic 4 might be about lasso-regression, where you have to check if the matrix is invertible to write its solution $w=(X^TX+\lambda I)^{-1}X^Ty$.

Topic seven is pretty useless, it regroupes terms about the format of the document.

Topic 10 could be about SVM, because of the kernel and classification terms.

In [31]:

#choose one document at random
np.random.seed(37)
doc_idx = np.random.randint(len(w))
#display its topic normalized distribution
print(gamma_mmse4[doc_idx]/np.sum(gamma_mmse4[doc_idx]))
#print the document itself
print(jmlr_papers[doc_idx]["abstract"])



[0.00454731 0.00454659 0.18200329 0.0045468  0.78162204 0.00454764
 0.0045461  0.00454668 0.00454643 0.00454713]
We show that, given data from a mixture of <i>k</i> well-separatedspherical Gaussians in <i>&#8476;<sup>d</sup></i>, a simple two-round variant ofEM will, with high probability, learn the parameters of the Gaussiansto near-optimal precision, if the dimension is high (<i>d >></i> ln<i>k</i>). We relate this to previous theoretical and empirical work on theEM algorithm.


This document seems to be linked mainly with only two topics, the third and fifth. It represents 96% of the text, and the rest of the topics all only represent about 0.4% of it.

Seeing that this abstract is about the EM-algorithm, itseems that our top-10 words previous analysis was incorrect even though the first topic included the word 'em' in its top.

**Q5.** Open questions :

- ###  What are some limitations of the LDA model ? Can you imagine an improvement ?
> Les limites visibiles du modèle LDA sont : 
> >Comme nous l'avons dit, le modèle LDA ne prends pas en compte l'ordre des mots dans ses Bags of Words(BOW). Cette simplification peut entraîner une perte d'informations, notamment sur le contexte et la nuance des propos.
> >
> >La LDA semble être un modèle coûteux en termes de calcul, surtout pour de grands ensembles de données. L'inférence peut être lente, et le modèle peut également être sensible aux paramètres d'initialisation, notamment le nombre de sujets K dont le choix n'est pas trivial et peut nécessiter une approche d'essai et d'erreur.
>
> Comme amélioration on pourrait suggérer : 
> > Utiliser des méthodes d'inférence plus efficaces/avancées, comme les réseaux de neurones pour capturer des relations plus complexes dans les données textuelles.
>>
> > Introduire des hiérarchies dans les sujets pour une représentation plus riche et plus nuancée des données.
> >

- ###  In this notebook, we have treated the hyperparameters as fixed. How could they be learned ?

- ### Can you imagine a method to choose the number of topics ?

- ### What strategies should we use to make the algorithm more efficient ?

**BONUS.** Papier-crayon. À partir du modèle, pouvez-vous dériver les lois conditionnelles de l'échantillonneur de Gibbs ? Pour rappel, nous avons besoin de ces lois pour dériver ensuite les updates de l'algorithme CAVI.